# MAIN CODE
아이폰 13

In [1]:
import requests
import datetime

thunder1=[] #name
thunder2=[] #price
location = []
description = []
time = []
status = []

for page_number in range(0,167):  #167

    
    
    url = "https://api.bunjang.co.kr/api/1/find_v2.json?q=%EC%95%84%EC%9D%B4%ED%8F%B013&order=score&page={0}&request_id=2022523143317&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4".format(page_number)
    res = requests.get(url)
    res.encoding=None
    data = res.json()
    
    for d in data['list']:
        name = d['name']
        loc = d['location']
        price = int(d['price'])
        pid = d['pid']
        unix = str(datetime.datetime.fromtimestamp(d['update_time']))
        unix = unix[0:7]
        sta = d['status']

        
        excepts = ["케이스","삽니다", "매입", "구매", "구입","프로","Pro","pro", "미니", "PRO", "13PRO" , '아이폰7', 'SE', 'se', '부품', 'Se', 'XS', 'max']
        targets = ['아이폰13', '아이폰 13']

        if (price >= 50000) & (price < 1600000) ==True :
            if ( d['ad']==False ):
                if any(x in d["name"] for x in targets) and not any(x in d['name'] for x in excepts):
                    thunder1.append(d['name'])
                    thunder2.append(price)
                    time.append(unix)
                    loc = loc.split(' ')
                    loc = loc[0]
                    # 위치
                    if len(loc) == 0:
                        location.append('NULL')
                    else:
                        location.append(loc)
                    # 판매 여부
                    if sta == '0' :           # 0: 판매중    3 : 판매완료
                        status.append('n')  # n 판매중
                    else : status.append('y')  # y 판매완료(예약완료 포함)

                    
                
                    url2 = 'https://api.bunjang.co.kr/api/1/product/{0}/detail_info.json?version=4'.format(pid)
                    res = requests.get(url2)
                    res.encoding=None
                    data = res.json()

                    try:
                        description.append(data['item_info']['description'])
                    except:
                        description.append('NULL')


                    # print(name, price, unix, loc, sta)

        else:
            pass

# print(thunder1,thunder2, time, location )


KeyboardInterrupt: 

# Desc 에서 배터리 효율 뽑아내기

In [ ]:
import re
from bs4 import BeautifulSoup

li = []



for i in range(len(description)):
    text = description[i]
    regex = '배터리\s*.*\d{2,3}'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    text = str(res)
    res2 = pattern2.findall(text)
    if len(res2) == 0:
        li.append('NULL')
    else:
        li.append(res2[0])

print(li)




['NULL', 'NULL', 'NULL', '100', 'NULL', 'NULL', 'NULL', '100', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '100', '100', 'NULL', 'NULL', '98', '100', 'NULL', 'NULL', '100', '100', 'NULL', '100', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '100', '100', 'NULL', 'NULL', 'NULL', 'NULL', '100', 'NULL', 'NULL', 'NULL', '98', 'NULL', 'NULL', '100', 'NULL', '100', 'NULL', '100', '100', '100', '100', '100', 'NULL', '100', 'NULL', 'NULL', '100', '100', '100', 'NULL', '99']


용량 빼내기

In [ ]:
size = []


for i in range(len(description)):
    text = description[i]
    text2 = thunder1[i]
    regex = '128|256|512'
    pattern =  re.compile(regex)
    res = pattern.findall(text)   #상품설명에서 용량 찾기
    res_size = pattern.findall(text2)  #네임에서 용량 찾기
    
    if len(res) == 0:
        if len(res_size) == 0 :
            size.append('NULL')
        else:
            size.append(res_size[0])
    else:
        size.append(res[0])
print(size)

['256', '512', '128', '128', '256', '128', '128', '128', '128', '128', '128', '128', '128', '512', '256', '128', '128', '256', '128', '128', '128', '128', '512', '256', '128', '128', '128', '128', '128', 'NULL', '128', '128', '256', '128', '128', '128', 'NULL', '256', '512', '128', '128', '256', '128', '512', 'NULL', '256', '128', '128', '128', '128', '128', '256', '256', '256', '512', '128', '128', 'NULL', '128', '128', '512', '128', 'NULL']


# 판다스로 데이터프레임 만들기

In [ ]:
import pandas as pd
df = pd.DataFrame({'name' : thunder1,
                 'price' : thunder2,
                 'time' : time,
                  'loction' : location,
                  'battery' : li,
                  'size': size,
                  'status' : status
                  })
df

,name,price,time,loction,battery,size,status
0,아이폰13 256기가 블랙 SS급 판매합니다)부산중고폰 0073,960000,2022-05,부산광역시,NULL,256,n
1,미개봉)아이폰13 512G 스타라이트(자급제),1200000,2022-05,서울특별시,NULL,512,n
2,미개봉)아이폰13 128G 자급제(스타라이트),950000,2022-05,서울특별시,NULL,128,n
3,아이폰13 핑크 128기가,850000,2022-05,NULL,100,128,n
4,미개봉)아이폰13 256G 자급제(스타라이트),1050000,2022-05,서울특별시,NULL,256,n
...,...,...,...,...,...,...,...
58,아이폰13 128GB 레드 S급 자급제,870000,2022-05,서울특별시,100,128,n
59,애플 아이폰13 128GB 블루 팝니다 울산중고폰,850000,2022-05,NULL,100,128,n
60,아이폰13 512 레드,1000000,2022-05,충청북도,100,512,n
61,아이폰13 128gb 미드나이트 미개봉 새제품 자급제,945000,2022-05,대전광역시,NULL,128,n


# 데이터 엑셀파일로 추출

In [ ]:
from pandas import ExcelWriter
df.to_excel('C:/Users/admin/Desktop/big/thunder_iphone13.xlsx', index=False)